# Bias in Machine Learning Schulung
### Ethischer Bias

### 1 Einführung

In diesem Teil der Übung soll der ethische Bias etwas näher betrachtet werden.    
Als Grundlage dient ein Dataset mit ~2 Millionen Kommentaren von Online-Plattformen.    
    
Mit diesen wird ein Modell trainiert, das anhand eines Kommentars bewerten soll, ob dieser "toxisch" oder "nicht toxisch" ist. Also ein Kommentar negative Ausdrucksweisen enthält oder nicht.
    
Da es hier nicht direkt im das Trainieren des Modells gehen soll ist dies bereits vorgegeben.

#### 1.1 Dataset laden und vorbereiten
Führt den unten angegebenen Code aus. Da das Dataset sehr groß ist kann es bei der Verarbeitung etwas länger dauern.

In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer

# Seed setzen, um immer die gleichen Ergebnisse zu erhalten
np.random.seed(0)

# Daten einlesen und benötigte Spalten extrahieren
data = pd.read_csv("comments_data.csv")
comments = data["comment_text"]
target = (data["target"]>0.7).astype(int)

# Daten in Trainings- und Test-Sets aufteilen (70:30)
comments_train, comments_test, y_train, y_test = train_test_split(comments, target, test_size=0.30, stratify=target)

# Alle einzelnen Wörter aus allen Kommentaren extrahieren und Wörterbuch mit ihrer Häufigkeit erstellen
vectorizer = CountVectorizer()
vectorizer.fit(comments_train)

# Dictionary für das Trainings- und Test-Set mit der Anzahl der Wörter erstellen
X_train = vectorizer.transform(comments_train)
X_test = vectorizer.transform(comments_test)

#### 1.2 Daten betrachten
Nun wollen wir uns die erstellten Daten anschauen und lernen mit diesen zu arbeiten. Mit `vectorizer.vocabulary_` bekommt ihr das Wörterbuch mit der Häufigkeit der einzelnen Wörter als Dictionary zurück (Dies werdet ihr später noch benötigen).    
    
Wie oft kommt in unserem Dataset das Wort `love` und das Wort `dumbass` vor?

Gebt euch ein paar Kommentare aus der Series(Dataframe mit einer Spalte) `comments_train` aus. Würdet ihr von diesen welche als toxisch einstufen?

In [ ]:
print(comments_train.iloc[22])
print(comments_train.iloc[17])
print(comments_train.iloc[28])

### 2 Model trainieren

#### 2.1 Model trainieren und Genauigkeit überprüfen
Führt den unten angegebenen Code nun aus, um ein einfaches Model zu trainieren. Zusätzlich berechnen wir die Genauigkeit des Models auf den Testdaten.

In [ ]:
from sklearn.linear_model import LogisticRegression

classifier = LogisticRegression(max_iter=2000)
classifier.fit(X_train, y_train)
score = classifier.score(X_test, y_test)
print("Genauigkeit:", score)

Ziemlich genau 93% der Kommentare werden von unserem Model korrekt erkannt.

#### 2.2 Model ausprobieren
Versucht nun ein paar eigene Kommentare an das Model zu übergeben und von diesem die Klassifizierung zurück zu erhalten.    
    
Da die Kommentare nur auf englischer Sprache basieren müsst ihr natürlich auch einen englischen Kommentar verfassen.    
Beispiele:
 - "I love Data Mining"
 - "Corona is stupid"
 
Versucht unterschiedliche Kommentare zu testen.

In [ ]:
# Funktion, um einen String zu Klassifizieren (Hier nichts verändern)
def classify_string(string, investigate=False):
    prediction = classifier.predict(vectorizer.transform([string]))[0]
    if prediction == 0:
        print("NICHT TOXISCH:", string)
    else:
        print("TOXISCH:", string)

In [ ]:
kommentar = "<Hier euer Kommentar>"
classify_string(mein_kommentar)

### 3 Genauere Betrachtung des Modells

Nachdem ihr das Modell nun etwas getestet habt, betrachten wir das Modell einmal etwas näher, um zu verstehen wie dieses Entscheidungen trifft.    
    
Das Modell fügt jedem der einzelnen Wörter in den Kommentaren einen Koeffizienten hinzu. Dieser sagt aus, ob ein entsprechendes Wort toxisch oder eben nicht toxisch ist. Desto höher der Koeffizient ist, desto mehr nimmt das Modell an, dass der Kommentar negativ behaftet ist.    
Ihr sollt nun ein Dataframe erstellen, das die Wörter mit den höchsten Koeffizienten anzeigt.

Benötigte Objekte:
 - `vectorizer.vocabulary_`: Gibt alle Wörter und ihre Häufigkeit als Dictionary zurück(Dies kennt ihr bereits). Wir benötigen nur die Wörter-Spalte alphabetisch sortiert als Liste.
 - `classifier.coef_`: Gibt ein mehrdimensionales Array bestehend aus den Koeffizienten zurück. Wir benötigen nur das erste Array.    
     
Erstellt nun ein Dataframe mit zwei Spalten: Einmal Spalte "Wort" mit den Wörtern und einmal die Spalte "Koeffizient" mit den entsprechenden Werten.    
    
**Hilfestellungen für die Wörter-Spalte:** 
  - <dict>.keys() gibt nur die Keys aus einem Dictionary zurück
  - list() wandelt die Keys in eine Liste um
  - sorted() sortiert die Liste alphabetisch (Damit die Einträge in der Liste zu den Koeffizienten passen muss diese so sortiert werden)


In [ ]:
# Spalte für die Wörter und Spalte für die Koeffizienten erstellen
words = #Wir benötigen nur die Wörter-Spalte alphabetisch sortiert als Liste
coeffs = #Wir benötigen nur das erste Array
# Dataframe erstellen
df = pd.DataFrame({"Wort": words, "Koeffizient": coeffs})

Sortiert nun das Dataframe absteigend nach der Spalte Koeffizient und lasst euch die ersten 15 Datensätze anzeigen.    
**Tipp:** Die Funktion `sort_values()`von pandas kann euch dabei helfen: [Dokumentation](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.sort_values.html)

In [ ]:
# Dataframe nach Koeffizient absteigend sortieren
df_sorted = 

# Erste 15 Datensätze des sortierten Dataframes ausgeben


Schaut euch die Wörter an. Gibt es Wörter die überraschend sind? Sind bei euch welche dabei, die nicht dabei sein sollten?

### 4 Bias finden

#### 4.1 Kommentare genauer analysieren
Wir wollen nun versuchen einen Bias in unserem Modell zu finden. Versucht dafür als erstes den Kommentar `"I have a christian friend"` zu klassifizieren.    
Nutzt dafür wieder einfach unsere Methode `classify_string()`

Probiert das gleiche mit dem Kommentar `"I have a muslim friend"`. Gibt es einen Unterschied?

Testet dieses Vorgehen nun mit weiteren Kommentaren. Beispielweise könnt ihr "I have a white bzw. black friend" testen, aber versucht euch gerne etwas eigenes zu überlegen.

#### Optionale Zusatzaufgabe

Lasst euch nun die Koeffizienten der einzelnen Wörter in den Kommentaren ausgeben, um herauszufinden welche Wörte wie bewertet wurden. Dafür könnt ihr euer zuvor erstelltes Dataframe mit der Liste aller Wörter und ihren Koeffizienten nutzen(`df_sorted`).    
    
**Hilfestellungen:**
 - `<string>.split()`: Teilt einen String in eine Liste der einzelnen Wörter (Trennung bei den Leerzeichen)
 - Um den Eintrag im Dataframe zu finden gibt es verschiedene Möglichkeiten: `<df>.<spalte>.isin(<liste der Wörter>)` gibt euch eine Series zurück bei der der es für jeden Datensatz einen boolschen Wert gibt und mit True bzw. False angibt, ob das Wort in dem Satz vorkommt. Dies könnt ihr auf das Dataset anwenden (Als Bedingung) um nur die Koeffizienten des Kommentars anzeigen zu lassen.

Versucht als nächstes den Kommentar "I have a muslim friend" auf die gleiche Weise zu bewerten.

#### 4.2 Bias identifizieren und verstehen
Nun wollen wir bewerten was wir gerade herausgefunden haben.
  
##### Wie klassifiziert das Modell eure Kommentare?    

**Antwort:** 

##### Konntet ihr einen potenziellen Bias in dem Model finden? 

**Antwort:** 

##### Stellt eine Vermutung an wieso das Model einen Bias aufweist.    

**Antwort:**    


##### Um welche Art von Bias könnte es sich handeln?

**Antwort:**    